Based off of work in vocab.ipynb, this program 

In [ ]:
from collections import Counter
import os
import pandas as pd
from gensim.models import Phrases, phrases

def getTexts(folder,searchList):
    fileToText = {}
    underscores = {}
    for root,dirs,files in os.walk(folder):
        for file in files:
                if '.txt' not in file: continue
                path = os.path.join(folder,file)
                f = open(path,'r')
                text = f.readlines()[0]
                if '_' in file: 
                        name = file.split('_')[0]
                        if name not in searchList: continue
                        if name not in underscores.keys(): 
                                underscores[name] = text
                        else: underscores[name] = underscores[name] + ' ' + text
                else: 
                        name = file.split('.')[0]
                        if name not in searchList: continue
                        fileToText[name] = text
                f.close()
        for name,text in underscores.items():
            fileToText[name] = text
        return fileToText

def collectgrams(keylist, searchword, collectlist):
        for key in keylist:
                if searchword in key.split('_'):
                        collectlist.append(key) 

def getgrams(txt):
    infile = open(txt,'r')
    lines = infile.readlines()
    infile.close()
    allgrams = []
    for line in lines: 
        line = line.split(':')
        ngrams = line[1].strip()
        if ngrams == '': continue
        ngrams = ngrams.strip().split(' ')
        for n in ngrams:
            allgrams.append(n)
    return allgrams 

#change for each time period: index is period - 1
period = open('/srv/data/timeranges.txt', 'r').readlines()[0]
period = period.strip().strip('[').strip(']').replace("'", '').split(', ')

bigramdata = getTexts('/srv/data/relevantEPBodyNOSTOP', period)
bigramtexts = list(bigramdata.values())
bigramnames = list(bigramdata.keys())

In [ ]:
#specify tobacco/drugs searchword list:
searchwords = ['tobacco']
# searchwords = ['drug', 'apothecary', 'confection', 'elixir', 'medicinable', 'medicine', 'arsenic', 'medecine', 'medicament', 'poppy', 'chemic', 'medicinal', 'intoxicate', 'potion', 'mithridate', 'antimony', 'opiate', 'opium']

remainders = [0, 1, 2]
for n in remainders:
    training = []
    testing = []
    for t in bigramtexts:
        #separating by thirds, remainders: 0, 1, 2
        if bigramtexts.index(t) % 3 == n:
            testing.append(t)
        else: 
            words = t.split(' ')
            training.append(words)
    print('section', n+1, '| training:', len(training), '| testing:', len(testing))

    bigrammodel = Phrases(training, min_count=1, threshold=-0.5, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
    trigrammodel = Phrases(bigrammodel[training], min_count=1, threshold=-0.5, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS )
    print('models trained')

    count = 0
    outfile = open('/srv/data/joy/period1tobacco.txt', 'a+')
    for text in testing:
        #for outputting to txt file, specify here
        name = bigramnames[bigramtexts.index(text)]
        testtext = text.strip().split(' ')

        grams = []
        bgcount = Counter(b for b in bigrammodel[testtext] if len(b.split("_")) > 1)
        tgcount = Counter(t for t in trigrammodel[testtext] if len(t.split("_")) > 2)
        for searchword in searchwords: 
            collectgrams(dict(bgcount).keys(), searchword, grams) 
            collectgrams(dict(tgcount).keys(), searchword, grams)
        
        gramstring = ' '.join(grams)
        outfile.write(name+': '+gramstring+"\n")
        count += 1
        if count%20 == 0:
            print(count, "files processed")
    outfile.close()
print('processing complete')
